In [ ]:
#Pandas DataReaderをインストール
!pip install pandas_datareader

In [ ]:
#必要なパッケージをインポート
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt

#ワーニング抑止
%matplotlib inline
# pd.core.common.is_list_like = pd.api.types.is_list_like

In [ ]:
#期間設定
start = '2015-06-01'
end = '2020-07-01'

#日経225平均を取得
df = data.DataReader('^N225', 'yahoo', start, end)

In [ ]:
# ここからSQLITE３でDBを作ってみよう
import sqlite3

# DB、テーブル作成
dbname = 'STOCK.db'
conn = sqlite3.connect(dbname)
df.to_sql('N225', conn, if_exists = 'replace')
# Option => if_exists　: append, replace, fail
conn.close()

In [ ]:
# 検索SQL
sql = '''
select * from N225
where Date between '2020-05-01 00:00:00' and  '2020-05-31 00:00:00'
'''

In [ ]:
# SQLを実行して結果をデータフレームに格納する
conn = sqlite3.connect(dbname)
df_vlm = pd.read_sql(sql, conn)
conn.close()

In [ ]:
df_vlm

In [ ]:
import datetime

##------------------------------
## グラフ表示関数
##------------------------------
def show_graph(start, end):
    dbname = 'STOCK.db'
    sql = F"select * from N225 where Date between '{start} 00:00:00' and  '{end} 23:59:59'"

    #DBから日経225平均を取得
    conn = sqlite3.connect(dbname)
    df_tmp = pd.read_sql(sql, conn)
    conn.close()
    
    #DATE型のINDEX作成
    dateidx = []
    for d in df_tmp['Date']:
        formatted = datetime.datetime.strptime(d, "%Y-%m-%d %H:%M:%S")
        dateidx.append(formatted)
    
    df_tmp['DateIdx'] = dateidx
    df = df_tmp.set_index('DateIdx')

    #全行表示
    #pd.set_option('display.max_rows', None)
    #print(df)
    
    #X軸、Y軸のデータを抽出
    date = df.index
    price= df['Adj Close']

    #移動平均取得
    span01=5
    span02=25
    span03=75
    df['sma01'] = price.rolling(window=span01).mean()
    df['sma02'] = price.rolling(window=span02).mean()
    df['sma03'] = price.rolling(window=span03).mean()

    #グラフの大きさを指定
    plt.figure(figsize=(20,10))

    #株価グラフ
    plt.subplot(2,1,1)   # 縦分割数, 横分割数, 配置位置
    plt.plot(date, price, label='Close', color='#99b898')
    plt.plot(date,df['sma01'], label='Ave 5', color='#e84a5f')
    plt.plot(date,df['sma02'], label='Ave 25', color='#ff847c')
    plt.plot(date,df['sma03'], label='Ave 75', color='#feceab')
    plt.legend()

    #出来高グラフ
    plt.subplot(2,1,2)
    plt.bar(date, df['Volume'], label='Volume', color='grey')
    plt.legend()

In [ ]:
show_graph('2020-01-01', '2020-06-30')